IMPORTS

In [9]:
import numpy as np 
import pandas as pd 
import os 
import json 
from PIL import Image
import random 
import time 
import cv2 
import math 
import albumentations as A 
from scipy.spatial.transform import Rotation as R 

HELPER FUNCTIONS

In [2]:
def project_point_to_image(C,T,P): 
    P_H = np.array([[P[0]],[P[1]],[P[2]],[1]]) 
    T_H = T[:3,:4]  
    uv = C @ T_H @ P_H 
    uv = uv / uv[2] 
    uv = uv[:2] 
    uv = uv.reshape((2)) 
    return uv 

def project_point_list_to_image(C,T,P_list): 
    n = len(P_list)
    uv_list = []  
    for P in P_list: 
        uv = project_point_to_image(C,T,P) 
        uv_list.append(uv) 
    return uv_list   

def transform_pts(pts, T):  
    pts_transformed = [] 
    for pt in pts: 
        pt = pt.reshape(3,1) 
        pt = np.vstack((pt, 1))  
        pt_transformed = T @ pt  
        pts_transformed.append(pt_transformed[:3]) 
    return pts_transformed

def overlay_points_on_image(image, pixel_points, radius=5, color=(0, 0, 255), thickness=-1):
    """
    Overlays a list of pixel points on the input image.

    Parameters:
    - image: The input image (a NumPy array).
    - pixel_points: A list of 2D pixel coordinates [(x1, y1), (x2, y2), ...].
    - radius: The radius of the circle to draw around each point. Default is 5.
    - color: The color of the circle (BGR format). Default is red (0, 0, 255).
    - thickness: The thickness of the circle. Default is -1 to fill the circle.

    Returns:
    - The image with points overlaid.
    """
    # Iterate over each pixel point and overlay it on the image
    for point in pixel_points:
        if point is not None:  # Only overlay valid points
            x, y = int(point[0]), int(point[1])
            # Draw a filled circle at the pixel coordinates
            cv2.circle(image, (x, y), radius, color, thickness)

    return image

def compute_2D_gridpoints(N=10,s=0.1): 
    # N = num squares, s = side length  
    u = np.linspace(-s/2, +s/2, N+1) 
    v = np.linspace(-s/2, +s/2, N+1) 
    gridpoints = [] 
    for uu in u:
        for vv in v: 
            gridpoints.append(np.array([uu,vv,0])) 
    return gridpoints 

CLASS DEFINITIONS 

In [28]:
class datapoint:
    def __init__(self, metadata_filepath, pose_filepath, rgb_filepath, seg_png_filepath, seg_json_filepath):
        # Store the filepaths
        self.metadata_filepath = metadata_filepath
        self.pose_filepath = pose_filepath
        self.rgb_filepath = rgb_filepath
        self.seg_png_filepath = seg_png_filepath
        self.seg_json_filepath = seg_json_filepath
        
        self.read_files()
        self.read_pose_data() 
        # self.compute_keypoints() 

        # TODO: self.idx = get_index() # or given as input 

    def read_files(self): 
        # Read the actual data from files and store it
        self.metadata = self._read_json(self.metadata_filepath) if self.metadata_filepath else None
        self.pose = self._read_json(self.pose_filepath) if self.pose_filepath else None
        self.rgb = self._read_rgb(self.rgb_filepath) if self.rgb_filepath else None
        self.seg_png = self._read_segmentation_png(self.seg_png_filepath) if self.seg_png_filepath else None
        self.seg_json = self._read_segmentation_json(self.seg_json_filepath) if self.seg_json_filepath else None 

    def read_pose_data(self): 
        # read pose data from pose json file 
        self.cam_pose = np.array([
                            [1, 0, 0, 0],
                            [0, -1, 0, 0],
                            [0, 0, -1, 0],
                            [0, 0, 0, 1]
                        ]) # NOTE: cam pose from isaac sim appears to be offset 
        self.tag_pose = np.array(self.pose["tag"]).transpose()  
        self.tag_pose *= np.array([
                            [10,10,10,1],
                            [10,10,10,1],
                            [10,10,10,1],
                            [1,1,1,1]
                        ]) # rescale the tag, FIXME: avoid hardcoding tag scale value 
        self.light_pose = self.pose["light"]
        self.tag_xyzabc = np.hstack((np.array(self.tag_pose[:3,3]), R.from_matrix(self.tag_pose[:3,:3]).as_euler("xyz",degrees=True))) # tag position in world frame 

    def compute_keypoints(self): 
        # FIXME: avoid hardcoding and take in as arguments 
        # camera parameters 
        width = 640 
        height = 480 
        focal_length = 24.0 
        horiz_aperture = 20.955
        # Pixels are square so we can do:
        vert_aperture = height/width * horiz_aperture
        fov = 2 * math.atan(horiz_aperture / (2 * focal_length))
        # compute focal point and center
        fx = width * focal_length / horiz_aperture
        fy = height * focal_length / vert_aperture
        cx = width / 2
        cy = height /2 

        self.C = np.array([
            [fx,0,cx],
            [0,fy,cy],
            [0,0,1]
        ])

        s = 0.1 # side length of marker 
        keypoints_tag_frame = compute_2D_gridpoints(N=10, s=s) 

        # transformations 
        tf_w_t = self.tag_pose 
        tf_w_c = self.cam_pose 
        tf_c_w = np.linalg.inv(tf_w_c) 

        keypoints_world_frame = [] 
        for kp_t in keypoints_tag_frame: 
            kp_t_homog = np.hstack((kp_t,np.array([1]))).reshape(4,1)
            kp_w_homog = tf_w_t @ kp_t_homog 
            keypoints_world_frame.append(kp_w_homog[:3].reshape(3)) 

        self.keypoints_image_space = project_point_list_to_image(self.C,tf_c_w,keypoints_world_frame) 

    def _read_json(self, filepath):
        """Read and parse JSON files."""
        with open(filepath, 'r') as file:
            return json.load(file)

    def _read_rgb(self, filepath):
        """Placeholder for reading RGB image files."""
        return filepath  # Placeholder: returning the file path to avoid memory overload

    def _read_segmentation_png(self, filepath):
        """Placeholder for reading segmentation PNG image files."""
        return filepath  # Placeholder: returning the file path to avoid memory overload

    def _read_segmentation_json(self, filepath):
        """Read segmentation JSON files."""
        with open(filepath, 'r') as file:
            return json.load(file)

    def compute_diffusion_reflectance(self): 
        """Compute the diffuse reflection based on pose and metadata."""
        N = np.array(self.tag_pose)[:3,2] 
        L = np.array(self.light_pose)[:3,2] 
        V = np.array(self.cam_pose)[:3,2] 
        light_exposure = self.metadata["light"]["exposure"] 
        I_incident = 2**light_exposure 
        shininess = 1.0  # Placeholder value 
        self.diffuse_reflection = I_incident * max(np.dot(N, L), 0)

    def preprocess_seg_img(self):
        """
        Preprocesses the segmentation image by resizing and converting it to a binary mask based on tag color.
        """

        seg_img_path = self.seg_png_filepath 
        seg_json_path = self.seg_json_filepath 

        # Validate that the segmentation image file exists
        if not os.path.exists(seg_img_path):
            raise FileNotFoundError(f"Segmentation image file not found: {seg_img_path}")

        # Validate that the JSON file exists
        if not os.path.exists(seg_json_path):
            raise FileNotFoundError(f"Segmentation JSON file not found: {seg_json_path}")

        # Load the segmentation JSON data 
        with open(seg_json_path, 'r') as json_file:
            seg_json = json.load(json_file)

            # Find the tag color from the JSON data
            for key, val in seg_json.items(): 
                if val.get("class") == "tag0":  
                    # Convert the key (which is a string representing a tuple) into an actual tuple
                    tag_seg_color = tuple(map(int, key.strip('()').split(', ')))  # Convert string '(140, 25, 255, 255)' into a tuple (140, 25, 255, 255)
                    break
            else:
                # raise ValueError("Tag with class 'tag0' not found in JSON.")
                tag_seg_color = tuple([-1,-1,-1,-1]) # impossible color value # FIXME: this is a workaround which can be turned into something more elegant 

        # Load and resize the segmentation image
        seg_img = Image.open(seg_img_path)
        # new_size = (480, 270)
        # new_size = (480*2, 270*2)
        # seg_img_resized = seg_img.resize(new_size)
        seg_img_resized = seg_img

        # Convert the resized image to a NumPy array
        seg_img_resized = np.array(seg_img_resized)

        # Check if the image is RGB (3 channels) or RGBA (4 channels) or grayscale (1 channel)
        if len(seg_img_resized.shape) == 3:
            if seg_img_resized.shape[2] == 3:  # RGB image
                # Compare each pixel to the tag color (e.g., RGB triplet)
                seg_img_resized = np.all(seg_img_resized == tag_seg_color[:3], axis=-1)  # Create binary mask for RGB image
            elif seg_img_resized.shape[2] == 4:  # RGBA image
                # Compare each pixel to the tag color (RGBA)
                seg_img_resized = np.all(seg_img_resized == tag_seg_color, axis=-1)  # Create binary mask for RGBA image
        else:  # If it's a single channel (grayscale), use it directly
            seg_img_resized = seg_img_resized == tag_seg_color  # Compare pixel values directly

        # Convert the binary mask to uint8 type (0 or 1)
        seg_img_resized = (seg_img_resized).astype(np.uint8) * 255  # Multiply by 255 to match image range

        # Convert the binary mask back to an image
        seg_img_resized = Image.fromarray(seg_img_resized)

        return seg_img_resized

    def __repr__(self):
        """Custom representation for the datapoint object."""
        # return f"datapoint(metadata_filepath={self.metadata_filepath}, pose_filepath={self.pose_filepath}, rgb_filepath={self.rgb_filepath}, seg_png_filepath={self.seg_png_filepath}, seg_json_filepath={self.seg_json_filepath})"
        description = [
            f"lighting_exposure={self.metadata["light"]["exposure"]:.2f}",
            # f"lighting_color={str(self.metadata["light"]["color"]) }" # FIXME: reduce to two decimal places 
            f"lighting_color=({self.metadata["light"]["color"][0]:.2f},{self.metadata["light"]["color"][1]:.2f},{self.metadata["light"]["color"][2]:.2f})" # FIXME: reduce to two decimal places 
        ]
        return "\n".join(description) 

class DataProcessor:
    def __init__(self, data_folders, out_dir):
        self.data_folders = data_folders
        self.out_dir = out_dir
        self.datapoints = []
        self.datapoints_train = []
        self.datapoints_val = []

    def _get_files_in_subfolder(self, folder, file_extension=None):
        """Helper method to get files in a subfolder, with an optional file extension filter."""
        files_list = os.listdir(folder)
        if file_extension:
            files_list = [file for file in files_list if file.endswith(file_extension)]
        # Order files_list by date created
        files_list = sorted(files_list, key=lambda x: os.path.getctime(os.path.join(folder, x)))  # Assumes creation dates are synchronized
        return files_list

    def process_folders(self):
        """Process the folders and create datapoint objects."""
        for data_folder in self.data_folders:
            metadata_subfolder = os.path.join(data_folder, "metadata")
            pose_subfolder = os.path.join(data_folder, "pose")
            rgb_subfolder = os.path.join(data_folder, "rgb")
            seg_subfolder = os.path.join(data_folder, "seg")

            # List files in subfolders 
            metadata_files = self._get_files_in_subfolder(metadata_subfolder, file_extension=".json")
            pose_files = self._get_files_in_subfolder(pose_subfolder, file_extension=".json")
            rgb_files = self._get_files_in_subfolder(rgb_subfolder, file_extension=".png")
            seg_png_files = self._get_files_in_subfolder(seg_subfolder, file_extension=".png")
            seg_json_files = self._get_files_in_subfolder(seg_subfolder, file_extension=".json")

            # Make sure the files are indexed and aligned properly (by index) across the subfolders
            max_length = max(len(metadata_files), len(pose_files), len(rgb_files), len(seg_png_files), len(seg_json_files))

            # Verify that the lengths are the same
            if not all(len(files) == max_length for files in [metadata_files, pose_files, rgb_files, seg_png_files, seg_json_files]):
                print(f"Lengths do not match for folder: {data_folder}")
                continue

            for i in range(max_length):
                # Use index 'i' to fetch corresponding files. If a file doesn't exist, use None.
                metadata_filepath = os.path.join(metadata_subfolder, metadata_files[i]) if i < len(metadata_files) else None
                pose_filepath = os.path.join(pose_subfolder, pose_files[i]) if i < len(pose_files) else None
                rgb_filepath = os.path.join(rgb_subfolder, rgb_files[i]) if i < len(rgb_files) else None
                seg_png_filepath = os.path.join(seg_subfolder, seg_png_files[i]) if i < len(seg_png_files) else None
                seg_json_filepath = os.path.join(seg_subfolder, seg_json_files[i]) if i < len(seg_json_files) else None

                # Create a datapoint object for each corresponding file
                data_point = datapoint(metadata_filepath, pose_filepath, rgb_filepath, seg_png_filepath, seg_json_filepath)
                self.datapoints.append(data_point)

    def get_datapoints(self):
        """Return the list of datapoint objects."""
        return self.datapoints
    
    def get_datapoints_filtered(self):
        """Return the list of filtered datapoint objects."""
        return self.datapoints_filtered 
    
    def check_image_okay(self, rgb_img, seg_img, min_tag_area=1000, min_tag_pix_mean=75, max_tag_pix_mean=250): 
        seg_img = np.array(seg_img) 
        # compute pixel area of tag segmentation 
        tag_pix_area = np.sum(seg_img == 255) 

        # create list of marker pixels using segmentation 
        marker_pixels = np.argwhere(seg_img == 255)  # Get the indices of pixels where the tag is present 
        # compute contrast of marker pixels using rgb image 
        rgb_img = np.array(rgb_img) 
        marker_rgb_values = rgb_img[marker_pixels[:, 0], marker_pixels[:, 1]]  # Get the RGB values of the marker pixels 
        marker_grey_values = np.mean(marker_rgb_values, axis=1)  # Compute the mean RGB values of the marker pixels 
        # compute contrast as the difference in magnitude of the RGB values of the marker pixels 
        tag_pix_contrast = marker_grey_values.max() - marker_grey_values.min()  
        tag_pix_mean = marker_grey_values.mean()
        if tag_pix_area > min_tag_area and tag_pix_mean > min_tag_pix_mean and tag_pix_mean < max_tag_pix_mean:  # FIXME: hardcoded threshold for tag area and diffuse reflection 
            bool_image_ok = True 
        else:
            bool_image_ok = False
        return bool_image_ok

    def filter_datapoints(self, min_tag_area=1000, min_tag_pix_mean=75, max_tag_pix_mean=250): 
        """Compute the diffusion reflectance and only keep datapoints with positive values."""
        self.datapoints_filtered = [] 
        self.datapoints_filtered_out = [] 
        for idx, dp in enumerate(self.datapoints):
            dp.compute_diffusion_reflectance() 
            seg_img = dp.preprocess_seg_img() 
            seg_img = np.array(seg_img) 
            # compute pixel area of tag segmentation 
            dp.tag_pix_area = np.sum(seg_img == 255) 
            self.datapoints[idx].tag_pix_area = np.sum(seg_img == 255) 

            # create list of marker pixels using segmentation 
            marker_pixels = np.argwhere(seg_img == 255)  # Get the indices of pixels where the tag is present 
            # compute contrast of marker pixels using rgb image 
            rgb_img = Image.open(dp.rgb_filepath) 
            rgb_img = np.array(rgb_img) 
            marker_rgb_values = rgb_img[marker_pixels[:, 0], marker_pixels[:, 1]]  # Get the RGB values of the marker pixels 
            marker_grey_values = np.mean(marker_rgb_values, axis=1)  # Compute the mean RGB values of the marker pixels 
            # compute contrast as the difference in magnitude of the RGB values of the marker pixels 
            dp.tag_pix_contrast = marker_grey_values.max() - marker_grey_values.min()  
            self.datapoints[idx].tag_pix_contrast = marker_grey_values.max() - marker_grey_values.min() 
            self.datapoints[idx].tag_pix_mean = marker_grey_values.mean()
            dp.tag_pix_mean = marker_grey_values.mean() 

            # os.makedirs(os.path.join(self.out_dir, "contrast"), exist_ok=True) 
            # # save contrast image for debugging purposes
            # cv2.imwrite(os.path.join(self.out_dir, "contrast", f"contrast_{idx}_{dp.tag_pix_contrast}_{dp.tag_pix_mean}.png"), rgb_img) 
            if self.check_image_okay(rgb_img, seg_img, min_tag_area=min_tag_area, min_tag_pix_mean=min_tag_pix_mean, max_tag_pix_mean=max_tag_pix_mean): 
                self.datapoints_filtered.append(dp)
            else: 
                self.datapoints_filtered_out.append(dp)

            # if dp.diffuse_reflection > min_diffuse_reflection and dp.tag_pix_area > min_tag_area and dp.tag_pix_mean > min_tag_pix_mean and dp.tag_pix_mean < max_tax_pix_mean:  # FIXME: hardcoded threshold for tag area and diffuse reflection 
            #     self.datapoints_filtered.append(dp) 
            # else: 
            #     self.datapoints_filtered_out.append(dp) 
                
    def split_train_val(self, filter=True, frac_train=0.8):
        """Split the datapoints into training and validation datasets."""
        if filter: 
            self.datapoints_train = random.sample(self.datapoints_filtered, int(frac_train * len(self.datapoints_filtered)))
            self.datapoints_val = [dp for dp in self.datapoints_filtered if dp not in self.datapoints_train]
        else:
            self.datapoints_train = random.sample(self.datapoints, int(frac_train * len(self.datapoints)))
            self.datapoints_val = [dp for dp in self.datapoints if dp not in self.datapoints_train]

    def create_directories(self):
        """Create directories for training and validation data."""
        dir_train = os.path.join(self.out_dir, "train")
        dir_val = os.path.join(self.out_dir, "val")
        dir_train_rgb = os.path.join(dir_train, "rgb")
        dir_train_seg = os.path.join(dir_train, "seg")
        dir_val_rgb = os.path.join(dir_val, "rgb")
        dir_val_seg = os.path.join(dir_val, "seg")

        os.makedirs(dir_train_rgb, exist_ok=True)
        os.makedirs(dir_train_seg, exist_ok=True)
        os.makedirs(dir_val_rgb, exist_ok=True)
        os.makedirs(dir_val_seg, exist_ok=True)

        return dir_train_rgb, dir_train_seg, dir_val_rgb, dir_val_seg

    def preprocess_rgb(self, img_path):  
        """Preprocess RGB image by resizing it."""
        # new_size = (480, 270)  # Define the new size
        # new_size = (480*2, 270*2)  # Define the new size
        img = Image.open(img_path)
        # img_resized = img.resize(new_size)
        img_resized = img 
        return img_resized

    def preprocess_seg_img(self, seg_img_path, seg_json_path, tag_seg_color=None):
        """
        Preprocesses the segmentation image by resizing and converting it to a binary mask based on tag color.
        """
        # Validate that the segmentation image file exists
        if not os.path.exists(seg_img_path):
            raise FileNotFoundError(f"Segmentation image file not found: {seg_img_path}")

        # Validate that the JSON file exists
        if not os.path.exists(seg_json_path):
            raise FileNotFoundError(f"Segmentation JSON file not found: {seg_json_path}")

        # Load the segmentation JSON data if tag_seg_color is not provided
        if tag_seg_color is None:
            with open(seg_json_path, 'r') as json_file:
                seg_json = json.load(json_file)

            # Find the tag color from the JSON data
            for key, val in seg_json.items(): 
                if val.get("class") == "tag0":  
                    # Convert the key (which is a string representing a tuple) into an actual tuple
                    tag_seg_color = tuple(map(int, key.strip('()').split(', ')))  # Convert string '(140, 25, 255, 255)' into a tuple (140, 25, 255, 255)
                    break
            else:
                # raise ValueError("Tag with class 'tag0' not found in JSON.")
                tag_seg_color = tuple([-1,-1,-1,-1]) # impossible color value # FIXME: this is a workaround which can be turned into something more elegant 

        # Load and resize the segmentation image
        seg_img = Image.open(seg_img_path)
        # new_size = (480, 270)
        # new_size = (480*2, 270*2)
        # seg_img_resized = seg_img.resize(new_size)
        seg_img_resized = seg_img

        # Convert the resized image to a NumPy array
        seg_img_resized = np.array(seg_img_resized)

        # Check if the image is RGB (3 channels) or RGBA (4 channels) or grayscale (1 channel)
        if len(seg_img_resized.shape) == 3:
            if seg_img_resized.shape[2] == 3:  # RGB image
                # Compare each pixel to the tag color (e.g., RGB triplet)
                seg_img_resized = np.all(seg_img_resized == tag_seg_color[:3], axis=-1)  # Create binary mask for RGB image
            elif seg_img_resized.shape[2] == 4:  # RGBA image
                # Compare each pixel to the tag color (RGBA)
                seg_img_resized = np.all(seg_img_resized == tag_seg_color, axis=-1)  # Create binary mask for RGBA image
        else:  # If it's a single channel (grayscale), use it directly
            seg_img_resized = seg_img_resized == tag_seg_color  # Compare pixel values directly

        # Convert the binary mask to uint8 type (0 or 1)
        seg_img_resized = (seg_img_resized).astype(np.uint8) * 255  # Multiply by 255 to match image range

        # Convert the binary mask back to an image
        seg_img_resized = Image.fromarray(seg_img_resized)

        return seg_img_resized

    def save_preprocessed_images(self, frac_train=0.8, augmentation=True):
        """Loop through train and val datapoints and save preprocessed images and segmentation masks."""
        dir_train_rgb, dir_train_seg, dir_val_rgb, dir_val_seg = self.create_directories()

        if augmentation: 
            transform = A.Compose([
                A.RandomShadow(shadow_roi=(0,0,1,1), num_shadows_limit=(1,2), shadow_dimension=4, shadow_intensity_range =(0.5, 0.8), p=0.8),  # Apply random shadows to the image
                A.RandomSunFlare(flare_roi=(0,0,1,1), num_flare_circles_range=(10,50), src_radius=100, src_color=(150,150,150), method="physics_based", p=0.8),  # Apply random sun flare to the image, TODO: come back to this, get labels 
                A.GaussNoise(var_limit=(0,0.05), per_channel=True, p=1),  # Add noise to the image 
                A.AdvancedBlur(blur_limit=(5,25), p=0.8),  # Apply blur to the image 
                A.RandomGamma(gamma_limit=(80, 120), p=0.8),  # Apply gamma correction to the image
                A.RandomBrightnessContrast(brightness_limit=(-0.25,0.25), contrast_limit=(-0.95,0.95), p=0.8),  # Adjust brightness and contrast
                A.ISONoise(intensity=(0.1, 0.5), color_shift=(0.01, 0.05), p=0.8),  # Apply ISO noise to the image 
            ])

        for i, dp in enumerate(self.datapoints_train): 
            img = self.preprocess_rgb(dp.rgb_filepath) 
            seg = self.preprocess_seg_img(dp.seg_png_filepath, dp.seg_json_filepath)   
            if augmentation: 
                augmented_img = Image.fromarray(transform(image=np.array(img)[:,:,:3])['image']) 
                while not self.check_image_okay(augmented_img, seg, min_tag_area=1000, min_tag_pix_mean=75, max_tag_pix_mean=250): 
                    augmented_img = Image.fromarray(transform(image=np.array(img)[:,:,:3])['image']) 
                # img = Image.fromarray(transform(image=np.array(img)[:,:,:3])['image'])       
                img = augmented_img 

            img.save(os.path.join(dir_train_rgb, f"img_{i}.png")) 
            seg.save(os.path.join(dir_train_seg, f"seg_{i}.png"))

        for i, dp in enumerate(self.datapoints_val):
            img = self.preprocess_rgb(dp.rgb_filepath) 
            seg = self.preprocess_seg_img(dp.seg_png_filepath, dp.seg_json_filepath) 
            if augmentation: 
                img = Image.fromarray(transform(image=np.array(img)[:,:,:3])['image'])       
            img.save(os.path.join(dir_val_rgb, f"img_{i}.png")) 
            seg.save(os.path.join(dir_val_seg, f"seg_{i}.png"))


In [ ]:
data_folders = [
    "/home/anegi/abhay_ws/marker_detection_failure_recovery/output/markers_20250305-165003",
    "/home/anegi/abhay_ws/marker_detection_failure_recovery/output/markers_20250310-005350/", 
]

# define OUT_DIR based on current date and time 
OUT_DIR = f"/home/anegi/abhay_ws/marker_detection_failure_recovery/segmentation_model/data/data_{time.strftime('%Y%m%d-%H%M%S')}"
# OUT_DIR = f"C:/Users/NegiA/Desktop/abhay_ws/marker_detection_failure_recovery/segmentation_model/sim_data/markers_20250314-181037/outputs/data_{time.strftime('%Y%m%d-%H%M%S')}"
# OUT_DIR = f"/media/rp/Elements/abhay_ws/marker_detection_failure_recovery/data/marker_obj_sdg/markers_20250305-165003/segmentation_model_data_{time.strftime('%Y%m%d-%H%M%S')}"
os.makedirs(OUT_DIR, exist_ok=True) 

# Create an instance of the DataProcessor class
processor = DataProcessor(data_folders, OUT_DIR)

# Process the folders to create the datapoint list
processor.process_folders()

# Retrieve and print length of the datapoints before and after filtering 
print(f"Number of datapoints: {len(processor.datapoints)}") 
processor.filter_datapoints() 
print(f"Number of filtered datapoints: {len(processor.datapoints_filtered)}")  

# Retrieve filtered datapoints
datapoints = processor.get_datapoints_filtered()

# Split the datapoints into training and validation sets
frac_train = 0.8
processor.split_train_val(filter=True, frac_train=frac_train) 
processor.save_preprocessed_images()


Number of datapoints: 1001
Number of filtered datapoints: 669
